In [ ]:
# 단계 1: 폰트 설치
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null



fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설치

# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설치

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Tuple
from pathlib import Path

class DataProcessor:
    """쌀 생산량과 기후 데이터를 처리하는 클래스"""

    CLIMATE_COLUMNS = {
        'annual': {
            '평균기온(°C)합계': '평균기온(°C)',
            '강수량(mm)': '합계 강수량(mm)',
            '평균 상대습도(%)합계': '평균 상대습도(%)',
            '일조시간(hr)': '합계 일조시간(hr)'
        },
        'monthly': {
            '평균기온(°C)': '평균기온(°C)',
            '평균상대습도(%)': '평균 상대습도(%)',
            '월합강수량(00~24h만)(mm)': '합계 강수량(mm)',
            '합계 일조시간(hr)': '합계 일조시간(hr)'
        }
    }

    REGION_MAPPING = {
        # 전라남도 지역
        '목포': '전라남도', '여수': '전라남도', '순천': '전라남도',
        '완도': '전라남도', '진도(첨찰산)': '전라남도', '진도군': '전라남도',
        '해남': '전라남도', '고흥': '전라남도', '광양시': '전라남도',
        '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',

        # 충청남도 지역
        '서산': '충청남도', '천안': '충청남도', '보령': '충청남도',
        '부여': '충청남도', '홍성': '충청남도',

        # 전라북도 지역
        '전주': '전라북도', '군산': '전라북도', '부안': '전라북도',
        '임실': '전라북도', '정읍': '전라북도', '남원': '전라북도',
        '장수': '전라북도', '고창군': '전라북도', '순창군': '전라북도',

        # 경상북도 지역
        '포항': '경상북도', '안동': '경상북도', '상주': '경상북도',
        '울진': '경상북도', '봉화': '경상북도', '영주': '경상북도',
        '문경': '경상북도', '청송군': '경상북도', '영덕': '경상북도',
        '의성': '경상북도', '구미': '경상북도', '영천': '경상북도',
        '경주시': '경상북도'
    }

    def __init__(self, target_regions: List[str]):
        self.target_regions = target_regions

    def fill_missing_years(self, df: pd.DataFrame, start_year: int = 2008, end_year: int = 2023) -> pd.DataFrame:
        """
        2008-2023년 범위의 데이터를 채우는 함수
        누락된 연도의 데이터는 해당 지역의 평균값으로 채움
        월별 데이터('YYYY-MM'형식)와 연간 데이터('YYYY'형식) 모두 처리
        지점과 지점명은 원본 매핑 유지
        """
        climate_columns = ['평균기온(°C)', '합계 강수량(mm)', '평균 상대습도(%)', '합계 일조시간(hr)']
        result_df = []

        # 일시 컬럼이 월별 데이터(YYYY-MM) 형식인지 확인
        is_monthly = isinstance(df['일시'].iloc[0], str) and '-' in str(df['일시'].iloc[0])

        for location in df['지점명'].unique():
            # 해당 지역의 데이터만 필터링
            location_data = df[df['지점명'] == location].copy()

            # 해당 지점의 지점번호 가져오기
            location_code = location_data['지점'].iloc[0]

            if is_monthly:
                # 월별 데이터 처리
                existing_dates = location_data['일시'].unique()
                all_dates = []
                for year in range(start_year, end_year + 1):
                    for month in range(1, 13):
                        all_dates.append(f"{year}-{month:02d}")

                # 해당 지역의 각 기후 지표 평균 계산 (월별)
                location_means = {}
                for col in climate_columns:
                    if col in location_data.columns:
                        # 각 월별로 평균 계산
                        monthly_means = {}
                        for month in range(1, 13):
                            month_data = location_data[location_data['일시'].str.endswith(f'-{month:02d}')][col]
                            if not month_data.empty:
                                monthly_means[month] = month_data.mean()
                            else:
                                monthly_means[month] = location_data[col].mean()
                        location_means[col] = monthly_means

                # 누락된 날짜에 대한 데이터 생성
                missing_dates = [date for date in all_dates if date not in existing_dates]
                for date in missing_dates:
                    year, month = map(int, date.split('-'))
                    new_row = {
                        '지점': location_code,  # 정수형 지점 코드 유지
                        '지점명': location,
                        '일시': date
                    }
                    for col in climate_columns:
                        if col in location_data.columns:
                            new_row[col] = location_means[col][month]
                    result_df.append(new_row)
            else:
                # 연간 데이터 처리
                existing_years = location_data['일시'].astype(int).unique()
                all_years = list(range(start_year, end_year + 1))

                # 해당 지역의 각 기후 지표 평균 계산
                location_means = {
                    col: location_data[col].mean()
                    for col in climate_columns
                    if col in location_data.columns
                }

                # 누락된 연도에 대한 데이터 생성
                missing_years = [year for year in all_years if year not in existing_years]
                for year in missing_years:
                    new_row = {
                        '지점': location_code,  # 정수형 지점 코드 유지
                        '지점명': location,
                        '일시': year,
                        **location_means
                    }
                    result_df.append(new_row)

            # 기존 데이터도 결과에 추가
            result_df.extend(location_data.to_dict('records'))

        # 결과를 DataFrame으로 변환
        result_df = pd.DataFrame(result_df)

        # 지점을 정수형으로 변환
        result_df['지점'] = result_df['지점'].astype(int)

        # 정렬 및 인덱스 재설정
        result_df = result_df.sort_values(['지점명', '일시']).reset_index(drop=True)

        # 숫자형 컬럼들 반올림
        for col in climate_columns:
            if col in result_df.columns:
                result_df[col] = result_df[col].round(1)

        return result_df

    def process_rice_data(self, file_path: Path, output_path: Optional[Path] = None) -> pd.DataFrame:
        """쌀 생산량 데이터 처리"""
        try:
            raw_data = pd.read_excel(file_path)
        except Exception as e:
            raise ValueError(f"Excel 파일 읽기 실패: {str(e)}")

        years = [int(col) for col in raw_data.columns[1:] if str(col).isdigit()]
        unique_years = sorted(set(years))

        restructured_data = []
        for idx, row in raw_data.iloc[2:].iterrows():
            region_name = row.iloc[0]
            for i, year in enumerate(unique_years):
                area_idx = i * 2 + 1
                prod_idx = i * 2 + 2

                try:
                    area = self._convert_to_float(row.iloc[area_idx])
                    production = self._convert_to_float(row.iloc[prod_idx])

                    restructured_data.append({
                        '행정구역': region_name,
                        '일시': year,
                        '재배면적(ha)': area,
                        '생산량(톤)': production
                    })
                except (ValueError, IndexError):
                    continue

        processed_df = pd.DataFrame(restructured_data)
        processed_df = self._filter_and_sort_data(processed_df)

        if output_path:
            processed_df.to_excel(output_path, index=False)

        return processed_df

    def process_climate_data(
        self,
        annual_path: Path,
        monthly_path: Path
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """기후 데이터 처리"""
        annual_climate = pd.read_csv(annual_path, encoding='euc-kr')
        monthly_climate = pd.read_csv(monthly_path, encoding='euc-kr')

        # 시계열 데이터 완성
        annual_climate = self.fill_missing_years(annual_climate)
        monthly_climate = self.fill_missing_years(monthly_climate)

        # 컬럼명 변경 및 행정구역 정보 추가
        annual_climate = self._prepare_climate_data(annual_climate, 'annual')
        monthly_climate = self._prepare_climate_data(monthly_climate, 'monthly')

        # 결측치 처리
        annual_climate = self._fill_missing_climate_data(annual_climate)
        monthly_climate = self._fill_missing_climate_data(monthly_climate)

        return annual_climate, monthly_climate

    def _convert_to_float(self, value: any) -> float:
        """문자열이나 숫자를 float로 변환"""
        if pd.isna(value) or str(value) == '-':
            return 0.0
        return float(value)

    def _filter_and_sort_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """데이터 필터링 및 정렬"""
        if self.target_regions:
            df = df[df['행정구역'].isin(self.target_regions)]
        return df.sort_values(['행정구역', '일시']).reset_index(drop=True)

    def _prepare_climate_data(self, df: pd.DataFrame, data_type: str) -> pd.DataFrame:
        """기후 데이터 전처리"""
        df = df.rename(columns=self.CLIMATE_COLUMNS[data_type])
        df['행정구역'] = df['지점명'].map(self.REGION_MAPPING)
        return df[df['행정구역'].isin(self.target_regions)]

    def _fill_missing_climate_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """기후 데이터 결측치 처리"""
        climate_columns = ['평균기온(°C)', '합계 강수량(mm)',
                         '평균 상대습도(%)', '합계 일조시간(hr)']

        df_filled = df.copy()
        for col in climate_columns:
            if col in df.columns:
                df_filled[col] = pd.to_numeric(
                    df_filled[col].replace('', np.nan),
                    errors='coerce'
                )
                region_means = df_filled.groupby('행정구역')[col].transform('mean').round(1)
                df_filled[col] = df_filled[col].fillna(region_means).round(1)

        return df_filled

def main():
    """메인 실행 함수"""
    # 파일 경로 설정
    rice_file = Path('/content/시군별_논벼_생산량.xlsx')
    annual_climate_file = Path('/content/연간기후전체지역.csv')
    monthly_climate_file = Path('/content/월별기후전체지역.csv')

    # 타겟 지역 설정
    target_regions = ['전라남도', '충청남도', '전라북도', '경상북도']

    # 데이터 프로세서 인스턴스 생성
    processor = DataProcessor(target_regions)

    try:
        # 1. 쌀 생산량 데이터 처리
        rice_data = processor.process_rice_data(
            rice_file,
            Path('filtered_rice_production.xlsx')
        )

        # 2. 기후 데이터 처리
        annual_climate, monthly_climate = processor.process_climate_data(
            annual_climate_file,
            monthly_climate_file
        )

        # 3. 처리된 기후 데이터 저장
        annual_climate.to_csv('filtered_climate_annual_data.csv',
                            index=False, encoding='utf-8-sig')
        monthly_climate.to_csv('filtered_climate_monthly_data.csv',
                             index=False, encoding='utf-8-sig')

        print("모든 데이터 처리가 완료되었습니다.")

    except Exception as e:
        print(f"처리 중 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

#전처리 완료

모든 데이터 처리가 완료되었습니다.
